In [8]:

from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import label_binarize
from itertools import cycle

import warnings
import os
import time
import math
import requests

import pandas as pd
import numpy as np

warnings.filterwarnings('ignore')

In [9]:
df = pd.read_csv('EPL_Data_featured.csv')

In [10]:
df.head()

,week,date,home_team,home_xg,score,away_xg,away_team,referee,game_id,home_team_elo,...,away_goals_scored_to_date,home_goals_conceded_to_date,away_goals_conceded_to_date,home_match_points,away_match_points,home_points_to_date,away_points_to_date,home_form,away_form,match_result
0,1,2018-08-10,manchester utd,1.5,2-1,1.8,leicester city,Andre Marriner,3ae83896,1884.934448,...,0,0,0,3,0,0,0,0.0,0.0,0
1,1,2018-08-11,bournemouth,2.2,2-0,1.4,cardiff city,Kevin Friend,352f5574,1673.780518,...,0,0,0,3,0,0,0,0.0,0.0,0
2,1,2018-08-11,fulham,0.7,0-2,1.0,crystal palace,Mike Dean,30875601,1633.799683,...,0,0,0,0,3,0,0,0.0,0.0,2
3,1,2018-08-11,huddersfield,0.3,0-3,1.9,chelsea,Chris Kavanagh,d82a37a7,1567.101318,...,0,0,0,0,3,0,0,0.0,0.0,2
4,1,2018-08-11,newcastle utd,1.0,1-2,2.0,tottenham,Martin Atkinson,dce15e01,1670.871338,...,0,0,0,0,3,0,0,0.0,0.0,2


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2280 entries, 0 to 2279
Data columns (total 29 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   week                         2280 non-null   int64  
 1   date                         2280 non-null   object 
 2   home_team                    2280 non-null   object 
 3   home_xg                      2280 non-null   float64
 4   score                        2280 non-null   object 
 5   away_xg                      2280 non-null   float64
 6   away_team                    2280 non-null   object 
 7   referee                      2280 non-null   object 
 8   game_id                      2280 non-null   object 
 9   home_team_elo                2280 non-null   float64
 10  away_team_elo                2280 non-null   float64
 11  season                       2280 non-null   int64  
 12  home_xG_to_date              2280 non-null   float64
 13  away_xG_to_date   

In [12]:
df.columns

Index(['week', 'date', 'home_team', 'home_xg', 'score', 'away_xg', 'away_team',
       'referee', 'game_id', 'home_team_elo', 'away_team_elo', 'season',
       'home_xG_to_date', 'away_xG_to_date', 'home_xG_against_to_date',
       'away_xG_against_to_date', 'home_goals_scored', 'away_goals_scored',
       'home_goals_scored_to_date', 'away_goals_scored_to_date',
       'home_goals_conceded_to_date', 'away_goals_conceded_to_date',
       'home_match_points', 'away_match_points', 'home_points_to_date',
       'away_points_to_date', 'home_form', 'away_form', 'match_result'],
      dtype='object')

In [13]:
columns_to_drop = ['week', 'date', 'home_team','home_xg', 'score', 'away_xg','away_team','game_id',
                   'season','home_match_points', 'away_match_points',
                   'home_goals_scored', 'away_goals_scored']

In [14]:
df_dropped = df.drop(columns=columns_to_drop)

# Display the first few rows of the dataframe after dropping the columns
df_dropped.head()

,referee,home_team_elo,away_team_elo,home_xG_to_date,away_xG_to_date,home_xG_against_to_date,away_xG_against_to_date,home_goals_scored_to_date,away_goals_scored_to_date,home_goals_conceded_to_date,away_goals_conceded_to_date,home_points_to_date,away_points_to_date,home_form,away_form,match_result
0,Andre Marriner,1884.934448,1697.498169,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0
1,Kevin Friend,1673.780518,1576.490356,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0
2,Mike Dean,1633.799683,1692.951660,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,2
3,Chris Kavanagh,1567.101318,1837.004272,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,2
4,Martin Atkinson,1670.871338,1914.848877,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,2


In [15]:
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2280 entries, 0 to 2279
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   referee                      2280 non-null   object 
 1   home_team_elo                2280 non-null   float64
 2   away_team_elo                2280 non-null   float64
 3   home_xG_to_date              2280 non-null   float64
 4   away_xG_to_date              2280 non-null   float64
 5   home_xG_against_to_date      2280 non-null   float64
 6   away_xG_against_to_date      2280 non-null   float64
 7   home_goals_scored_to_date    2280 non-null   int64  
 8   away_goals_scored_to_date    2280 non-null   int64  
 9   home_goals_conceded_to_date  2280 non-null   int64  
 10  away_goals_conceded_to_date  2280 non-null   int64  
 11  home_points_to_date          2280 non-null   int64  
 12  away_points_to_date          2280 non-null   int64  
 13  home_form         

In [16]:

# Assuming df_dropped is your DataFrame after dropping columns

# Define the target variable and the feature set
X = df_dropped.drop('match_result', axis=1)
y = df_dropped['match_result']

# Identify categorical features for CatBoost
categorical_features_indices = list(X.select_dtypes(include=['object']).columns)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize CatBoostClassifier
catboost_model = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    depth=3,
    cat_features=categorical_features_indices,
    verbose=10
)

# Train CatBoostClassifier
catboost_model.fit(X_train, y_train)

# Predict on the testing set
y_pred = catboost_model.predict(X_test)

# Calculate the F1 score
f1 = f1_score(y_test, y_pred, average='weighted')  # Use 'weighted' if classes are imbalanced
print(f'The F1 score of the CatBoost model is: {f1}')


0:	learn: 1.0761518	total: 76.1ms	remaining: 7.53s
10:	learn: 0.9865637	total: 108ms	remaining: 871ms
20:	learn: 0.9569969	total: 135ms	remaining: 507ms
30:	learn: 0.9418076	total: 158ms	remaining: 351ms
40:	learn: 0.9334614	total: 180ms	remaining: 260ms
50:	learn: 0.9262121	total: 203ms	remaining: 195ms
60:	learn: 0.9213934	total: 225ms	remaining: 144ms
70:	learn: 0.9163160	total: 249ms	remaining: 102ms
80:	learn: 0.9097662	total: 272ms	remaining: 63.8ms
90:	learn: 0.9037178	total: 300ms	remaining: 29.7ms
99:	learn: 0.8990405	total: 324ms	remaining: 0us
The F1 score of the CatBoost model is: 0.47244475797107366


In [17]:



# Assuming X and y are already defined and categorical features are properly handled

# Create a CatBoost classifier instance
catboost = CatBoostClassifier(cat_features=categorical_features_indices, verbose=0)

# Define the pipeline
pipeline = Pipeline([
    # Here you can add other preprocessing steps if needed
    ('classifier', catboost)
])

# Define the hyperparameters grid to be tested
params_grid = {
    'classifier__iterations': [25, 50, 100],
    'classifier__depth': [5, 6, 7, 8],
    'classifier__learning_rate': [ 0.1, 0.2, 0.3, 0.4]
    # You can add other parameters you want to tune
}

# Setup the GridSearchCV object
grid_search = GridSearchCV(pipeline, params_grid, cv=5, scoring='f1_weighted', n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Best hyperparameters
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

# Predict on the testing set using the best estimator
y_pred = best_estimator.predict(X_test)

# Calculate the F1 score with the best estimator
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Best parameters: {best_params}')
print(f'Best F1 score: {f1}')


Best parameters: {'classifier__depth': 6, 'classifier__iterations': 25, 'classifier__learning_rate': 0.4}
Best F1 score: 0.49637389474958277


In [18]:
# Get feature importances from the best estimator
feature_importances = best_estimator.named_steps['classifier'].get_feature_importance()

# Get feature names
feature_names = X_train.columns

# Create a DataFrame to display feature importances
importances_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
})

# Sort the DataFrame to show the most important features at the top
importances_df.sort_values(by='Importance', ascending=False, inplace=True)

print(importances_df)


                        Feature  Importance
1                 home_team_elo   17.532386
2                 away_team_elo   14.644252
13                    home_form   11.542431
0                       referee   10.068952
14                    away_form    9.748997
7     home_goals_scored_to_date    6.159892
10  away_goals_conceded_to_date    5.557130
12          away_points_to_date    5.188257
4               away_xG_to_date    4.440570
6       away_xG_against_to_date    3.604443
5       home_xG_against_to_date    3.023212
11          home_points_to_date    2.824958
8     away_goals_scored_to_date    2.608878
3               home_xG_to_date    1.976406
9   home_goals_conceded_to_date    1.079236
